In [ ]:
# default_exp update

# Data Checks, Updates & Cleaning

<br>

### Imports

In [2]:
#exports
import json
import pandas as pd

import os
import typing
from warnings import warn

from powerdict import construct

In [3]:
from IPython.display import JSON

<br>

### User Inputs

In [4]:
intermediate_data_dir = '../data/intermediate'
definitions_dir = '../data/definitions'
updates_data_dir = '../data/updates'
output_data_dir = '../data/output'

<br>

### Loading in Intermediate Data

In [5]:
df = pd.read_csv(f'{intermediate_data_dir}/power_stations.csv').astype(str).set_index('osuked_id')

df.head()

,esail_id,fuel_type,sett_bmu_id,longitude,latitude
osuked_id,,,,,
10000,MARK,biomass,"E_MARK-1, E_MARK-2",-3.603516,57.480403
10001,DIDC,coal,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3",-1.26757,51.62363
10002,ABTH,coal,"T_ABTH7, T_ABTH8, T_ABTH9",-3.404866,51.387312
10003,COTPS,coal,"T_COTPS-1, T_COTPS-2, T_COTPS-3, T_COTPS-4",-0.648193,53.245495
10004,DRAXX,coal,"T_DRAXX-1, T_DRAXX-2, T_DRAXX-3, T_DRAXX-4, T_...",-0.626221,53.748711


<br>

### Custom Data Checks

We'll start by confirming that none of the plants have multiple locations assigned to them

In [6]:
#exports
def filter_cols_for_one_to_many(df, cols=['longitude', 'latitude']):
    df_one_to_many_filt = df[df[cols].astype(str).agg(''.join, axis=1).str.contains(', ')]
    return df_one_to_many_filt

In [7]:
df_multiple_locs = filter_cols_for_one_to_many(df)

assert df_multiple_locs.size == 0, 'There should not be multiple locations for a single site'

In [22]:
#exports
def apply_updates(df, updates_data_dir):
    update_files = [f for f in os.listdir(updates_data_dir) if f.replace('.json', '') in df.columns]

    for update_file in update_files:    
        with open(f'{updates_data_dir}/{update_file}', 'r') as f:
            update_dict = json.load(f)
            
        update_col = update_file.replace('.json', '')
        update_values = {k: v['new_value'] for k, v in update_dict.items()}
        
        df = construct.update_df_col(df, update_col, update_values)
        
    return df

In [23]:
df = apply_updates(df, updates_data_dir)

df.head()

,esail_id,fuel_type,sett_bmu_id,longitude,latitude
osuked_id,,,,,
10000,MARK,biomass,"E_MARK-1, E_MARK-2",-3.603516,57.480403
10001,DIDC,coal,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3",-1.267570,51.623630
10002,ABTH,coal,"T_ABTH7, T_ABTH8, T_ABTH9",-3.404866,51.387312
10003,COTPS,coal,"T_COTPS-1, T_COTPS-2, T_COTPS-3, T_COTPS-4",-0.648193,53.245495
10004,DRAXX,"coal, biomass","T_DRAXX-1, T_DRAXX-2, T_DRAXX-3, T_DRAXX-4, T_...",-0.626221,53.748711


<br>

### Checking/Applying Output Definitions

In [10]:
with open(f'{definitions_dir}/outputs.json', 'r') as f:
    output_defs = json.load(f)
    
JSON(output_defs)

<IPython.core.display.JSON object>

In [11]:
#exports
def check_output_col_values(df, output_col, output_defs):
    output_col_def = output_defs[output_col]
    
    if isinstance(output_col_def, list):
        output_col_elems = list()

        for elem in df[output_col].to_list():
            if ', ' in elem:
                output_col_elems += elem.split(', ')
            else:
                output_col_elems += [elem]

        unexpected_col_elems = sorted(list(set(output_col_elems) - set(output_col_def)))
        
        if len(unexpected_col_elems) > 0:
            warn(f"The following list elements were not expected in {output_col}: {', '.join(unexpected_col_elems)}")
        
    return

In [12]:
output_col = 'fuel_type'

check_output_col_values(df, output_col, output_defs)

In [13]:
#exports
def apply_output_col_types(df, output_col, output_defs):
    output_col_def = output_defs[output_col]
    
    if isinstance(output_col_def, list):
        dtype = type(output_col_def[0])
    else:
        dtype = output_col_def
        
    df[output_col] = df[output_col].astype(dtype)
        
    return df

In [14]:
output_col = 'longitude'

apply_output_col_types(df, output_col, output_defs)['longitude'].dtype

dtype('float64')

In [15]:
#exports
def check_and_apply_output_defs(df, definitions_dir):
    with open(f'{definitions_dir}/outputs.json', 'r') as f:
        output_defs = json.load(f)

    for output_col in output_defs.keys():
        if output_col not in df.columns:
            warn(f'{output_col} was not found in the output dataset')
        else:
            check_output_col_values(df, output_col, output_defs)
            df = apply_output_col_types(df, output_col, output_defs)
            
    return df

In [16]:
df = check_and_apply_output_defs(df, definitions_dir)

df.head()

<ipython-input-15-424d1db4ecaf>:7: UserWarning: plant_type was not found in the output dataset
  warn(f'{output_col} was not found in the output dataset')
<ipython-input-15-424d1db4ecaf>:7: UserWarning: capacity was not found in the output dataset
  warn(f'{output_col} was not found in the output dataset')


,esail_id,fuel_type,sett_bmu_id,longitude,latitude
osuked_id,,,,,
10000,MARK,biomass,"E_MARK-1, E_MARK-2",-3.603516,57.480403
10001,DIDC,coal,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3",-1.267570,51.623630
10002,ABTH,coal,"T_ABTH7, T_ABTH8, T_ABTH9",-3.404866,51.387312
10003,COTPS,coal,"T_COTPS-1, T_COTPS-2, T_COTPS-3, T_COTPS-4",-0.648193,53.245495
10004,DRAXX,"coal, biomass","T_DRAXX-1, T_DRAXX-2, T_DRAXX-3, T_DRAXX-4, T_...",-0.626221,53.748711


In [18]:
# then start doing some matching with GPPDB

In [27]:
#hide
from nbdev.export import *
notebook2script()

Converted 01-source-data.ipynb.
Converted 02-data-definitions.ipynb.
Converted 03-database-construction.ipynb.
Converted 04-updates-and-cleaning.ipynb.
Converted 05-pipeline.ipynb.
